# Objective















In [191]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import warnings
import os

In [125]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [126]:
df1 = pd.read_excel('../Data/case_study1.xlsx')
df2 = pd.read_excel('../Data/case_study2.xlsx')

In [128]:
df1.sample(10)

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,Total_TL_opened_L12M,Tot_TL_closed_L12M,pct_tl_open_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,Auto_TL,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
2207,2208,3,0,3,0,0,0.000,0.000,1.000,0.000,1,0,0.333,0.000,0,0,0,3,0,0,0,0,3,0,12,12
8786,8787,5,1,4,1,1,0.200,0.200,0.800,0.200,4,1,0.800,0.200,0,0,0,5,0,0,0,0,5,0,12,3
13157,13158,4,3,1,1,3,0.250,0.750,0.250,0.750,2,3,0.500,0.750,1,0,0,1,0,0,0,0,4,3,19,2
16039,16040,11,7,4,0,0,0.000,0.000,0.364,0.636,2,1,0.182,0.091,0,0,1,1,0,0,1,8,3,8,67,7
2734,2735,2,0,2,0,0,0.000,0.000,1.000,0.000,1,0,0.500,0.000,0,0,0,0,0,1,0,2,0,1,17,9
36522,36523,6,5,1,0,0,0.000,0.000,0.167,0.833,1,2,0.167,0.333,0,1,0,0,0,0,0,6,0,5,122,8
26959,26960,15,12,3,0,1,0.000,0.067,0.200,0.800,1,1,0.067,0.067,0,1,0,1,11,0,1,13,2,1,173,11
41819,41820,1,0,1,0,0,0.000,0.000,1.000,0.000,1,0,1.000,0.000,0,0,0,1,0,0,0,0,1,0,6,6
33246,33247,4,3,1,0,0,0.000,0.000,0.250,0.750,1,1,0.250,0.250,0,1,0,0,3,0,0,4,0,0,21,11
2935,2936,13,12,1,1,0,0.077,0.000,0.077,0.923,1,3,0.077,0.231,0,0,0,10,0,0,0,0,13,3,58,4


In [131]:
df1.isna().sum()

PROSPECTID              0
Total_TL                0
Tot_Closed_TL           0
Tot_Active_TL           0
Total_TL_opened_L6M     0
Tot_TL_closed_L6M       0
pct_tl_open_L6M         0
pct_tl_closed_L6M       0
pct_active_tl           0
pct_closed_tl           0
Total_TL_opened_L12M    0
Tot_TL_closed_L12M      0
pct_tl_open_L12M        0
pct_tl_closed_L12M      0
Tot_Missed_Pmnt         0
Auto_TL                 0
CC_TL                   0
Consumer_TL             0
Gold_TL                 0
Home_TL                 0
PL_TL                   0
Secured_TL              0
Unsecured_TL            0
Other_TL                0
Age_Oldest_TL           0
Age_Newest_TL           0
dtype: int64

**It is not possible to have zero null values in a real world dataset, we will look at this unusual behaviour later**

**Shape of Datasets**

In [65]:
df1.shape

(51336, 26)

In [66]:
df2.shape

(51336, 62)

**`Observations`**
- We have `51336` Prospect IDs, which is reflecting we have data of `51336` customers.
- `df1` has 26 features.
- Where as `df2` has 62 features.

**Data types of features and Count of Non Null values**

In [67]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PROSPECTID            51336 non-null  int64  
 1   Total_TL              51336 non-null  int64  
 2   Tot_Closed_TL         51336 non-null  int64  
 3   Tot_Active_TL         51336 non-null  int64  
 4   Total_TL_opened_L6M   51336 non-null  int64  
 5   Tot_TL_closed_L6M     51336 non-null  int64  
 6   pct_tl_open_L6M       51336 non-null  float64
 7   pct_tl_closed_L6M     51336 non-null  float64
 8   pct_active_tl         51336 non-null  float64
 9   pct_closed_tl         51336 non-null  float64
 10  Total_TL_opened_L12M  51336 non-null  int64  
 11  Tot_TL_closed_L12M    51336 non-null  int64  
 12  pct_tl_open_L12M      51336 non-null  float64
 13  pct_tl_closed_L12M    51336 non-null  float64
 14  Tot_Missed_Pmnt       51336 non-null  int64  
 15  Auto_TL            

In [68]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 62 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PROSPECTID                    51336 non-null  int64  
 1   time_since_recent_payment     51336 non-null  int64  
 2   time_since_first_deliquency   51336 non-null  int64  
 3   time_since_recent_deliquency  51336 non-null  int64  
 4   num_times_delinquent          51336 non-null  int64  
 5   max_delinquency_level         51336 non-null  int64  
 6   max_recent_level_of_deliq     51336 non-null  int64  
 7   num_deliq_6mts                51336 non-null  int64  
 8   num_deliq_12mts               51336 non-null  int64  
 9   num_deliq_6_12mts             51336 non-null  int64  
 10  max_deliq_6mts                51336 non-null  int64  
 11  max_deliq_12mts               51336 non-null  int64  
 12  num_times_30p_dpd             51336 non-null  int64  
 13  n

**`Observations`**
- `df1` has two data types for it's columns, int and float.
- `df2` int, float, object data types. with majority of features having int data type.

**`Doubt:`** Both the data sets are showing `0` null values, which is quite impossible with this kind of large and real world data, we will inquire the reason later.

In [69]:
df1.describe()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,Total_TL_opened_L12M,Tot_TL_closed_L12M,pct_tl_open_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,Auto_TL,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
count,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000
mean,25668.500000,4.858598,2.770415,2.088184,0.736754,0.428919,0.184574,0.089095,0.577542,0.422458,1.503701,0.736851,0.395184,0.149989,0.546751,0.593268,0.124981,1.136084,1.561847,0.070146,0.282511,2.844904,2.013694,1.089762,-32.575639,-62.149525
std,14819.571046,7.177116,5.941680,2.290774,1.296717,0.989972,0.297414,0.205635,0.379867,0.379867,2.119399,1.454120,0.391930,0.257267,1.085529,0.900585,0.505201,2.227997,5.376434,0.340861,0.858168,6.187177,3.198322,2.417496,2791.869609,2790.818622
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-99999.000000,-99999.000000
25%,12834.750000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,4.000000
50%,25668.500000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.556000,0.444000,1.000000,0.000000,0.333000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,33.000000,8.000000
75%,38502.250000,5.000000,3.000000,3.000000,1.000000,1.000000,0.308000,0.053000,1.000000,0.750000,2.000000,1.000000,0.750000,0.250000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,3.000000,2.000000,1.000000,64.000000,17.000000
max,51336.000000,235.000000,216.000000,47.000000,27.000000,19.000000,1.000000,1.000000,1.000000,1.000000,39.000000,39.000000,1.000000,1.000000,34.000000,27.000000,27.000000,41.000000,235.000000,10.000000,29.000000,235.000000,55.000000,80.000000,392.000000,392.000000


**`Observations`**
1. `Total_TL:` 75% of customers have less than 5 trade lines and 25% having more than 5 trade lines.
2. `Tot_Active_TL:` 50% of the customers have 1 active trade line and 50% have more than 1. Maximum number of active Trade lines is 47.
3. `Tot_Missed_Pmnt:` 50% of customers had never missed any payment.

**~Unusual Behaviour of data in `Age_Oldest_TL`	`Age_Newest_TL`**
- We have talked before, it is unusual to have 0 null values in real world data. As we can see `Age_Oldest_TL`	`Age_Newest_TL` both the columns have minimum value of `-99999` which is not realistic. Age of an account can not be in `-ve`
- After talking to data provider, i've come to know that, data engineer team imputed -99999 as the alternate to `Null` values for data integrity.    
- **Result of which, we will treat -99999 as a Null value.**

In [70]:
df2.describe()

,PROSPECTID,time_since_recent_payment,time_since_first_deliquency,time_since_recent_deliquency,num_times_delinquent,max_delinquency_level,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,max_deliq_6mts,max_deliq_12mts,num_times_30p_dpd,num_times_60p_dpd,num_std,num_std_6mts,num_std_12mts,num_sub,num_sub_6mts,num_sub_12mts,num_dbt,num_dbt_6mts,num_dbt_12mts,num_lss,num_lss_6mts,num_lss_12mts,recent_level_of_deliq,tot_enq,CC_enq,CC_enq_L6m,CC_enq_L12m,PL_enq,PL_enq_L6m,PL_enq_L12m,time_since_recent_enq,enq_L12m,enq_L6m,enq_L3m,AGE,NETMONTHLYINCOME,Time_With_Curr_Empr,pct_of_active_TLs_ever,pct_opened_TLs_L6m_of_L12m,pct_currentBal_all_TL,CC_utilization,CC_Flag,PL_utilization,PL_Flag,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,Credit_Score
count,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,5.133600e+04,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000,51336.000000
mean,25668.500000,-8129.961314,-70020.091320,-70022.375838,1.573749,-70003.987085,13.521953,0.184977,0.480053,0.295076,-25105.208587,-21092.727676,0.728884,0.426971,9.204671,1.484027,3.319444,0.062646,0.002435,0.009292,0.025947,0.001208,0.004130,0.015097,0.000974,0.002844,11.266090,-12308.234046,-12312.463866,-12312.734261,-12312.644869,-12311.878195,-12312.429270,-12312.209229,-12084.842099,-12310.233248,-12311.123987,-12311.784771,33.758532,2.642419e+04,110.707846,0.577542,0.302955,-139.416072,-92791.608990,0.089469,-86556.225194,0.167874,0.190414,0.065182,0.170492,0.056302,-45127.943635,0.271116,0.052887,679.859222
std,14819.571046,27749.328514,45823.312757,45819.820741,4.165012,45847.976100,53.336976,0.710240,1.522210,1.027471,43366.162767,40805.742595,2.762120,2.101404,21.308881,3.364581,7.518414,0.801218,0.091383,0.233403,0.635611,0.070331,0.179885,0.529730,0.075413,0.185349,46.100309,32860.363869,32858.778356,32858.677000,32858.710505,32858.997860,32858.791308,32858.873781,32946.916037,32859.614436,32859.280542,32859.032853,8.816364,2.002711e+04,76.046831,0.379867,0.406049,3742.613530,25861.225129,0.285423,34111.414750,0.373758,0.376218,0.235706,0.350209,0.213506,49795.784556,0.444540,0.223810,20.502764
min,1.000000,-99999.000000,-99999.000000,-99999.000000,0.000000,-99999.000000,0.000000,0.000000,0.000000,0.000000,-99999.000000,-99999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,21.000000,0.000000e+00,0.000000,0.000000,0.000000,-99999.000000,-99999.000000,0.000000,-99999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-99999.000000,0.000000,0.000000,469.000000
25%,12834.750000,46.000000,-99999.000000,-99999.000000,0.000000,-99999.000000,0.000000,0.000000,0.000000,0.000000,-99999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,27.000000,1.800000e+04,61.000000,0.250000,0.000000,0.083000,-99999.000000,0.000000,-99999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-99999.000000,0.000000,0.000000,669.00000

**`Observations`**
- In this datasets we have 62 features, out of that 21 features have `-99999` as null value.

## Treating Null Values

**Rows having Null values in `df1`.**

In [132]:
df1[df1['Age_Newest_TL'] == -99999].count()

PROSPECTID              40
Total_TL                40
Tot_Closed_TL           40
Tot_Active_TL           40
Total_TL_opened_L6M     40
Tot_TL_closed_L6M       40
pct_tl_open_L6M         40
pct_tl_closed_L6M       40
pct_active_tl           40
pct_closed_tl           40
Total_TL_opened_L12M    40
Tot_TL_closed_L12M      40
pct_tl_open_L12M        40
pct_tl_closed_L12M      40
Tot_Missed_Pmnt         40
Auto_TL                 40
CC_TL                   40
Consumer_TL             40
Gold_TL                 40
Home_TL                 40
PL_TL                   40
Secured_TL              40
Unsecured_TL            40
Other_TL                40
Age_Oldest_TL           40
Age_Newest_TL           40
dtype: int64

**`Observations`**
- We have 40 rows with `-99999` values, which is very few number as compared to 50k rows. We can remove these rows from the dataset. It won't effect the overall data because the ratio is so small.

In [133]:
df1 = df1[df1['Age_Newest_TL'] != -99999]

In [134]:
df1.describe()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,Total_TL_opened_L12M,Tot_TL_closed_L12M,pct_tl_open_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,Auto_TL,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
count,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000,51296.000000
mean,25668.574353,4.861471,2.772321,2.089149,0.737328,0.429254,0.184718,0.089164,0.577349,0.422651,1.504874,0.737426,0.395492,0.150106,0.546748,0.593711,0.125078,1.136970,1.563046,0.070181,0.282712,2.846752,2.014718,1.089773,45.376969,15.780022
std,14819.611579,7.179141,5.943536,2.291375,1.297059,0.990286,0.297486,0.205700,0.379804,0.379804,2.119809,1.454542,0.391927,0.257334,1.085863,0.900786,0.505386,2.228639,5.378357,0.340963,0.858461,6.189165,3.199333,2.418314,41.737151,21.995676
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12836.750000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,4.000000
50%,25666.500000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.556000,0.444000,1.000000,0.000000,0.333000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,33.000000,8.000000
75%,38503.250000,5.000000,3.000000,3.000000,1.000000,1.000000,0.312000,0.054000,1.000000,0.750000,2.000000,1.000000,0.750000,0.250000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,3.000000,2.000000,1.000000,64.000000,17.000000
max,51336.000000,235.000000,216.000000,47.000000,27.000000,19.000000,1.000000,1.000000,1.000000,1.000000,39.000000,39.000000,1.000000,1.000000,34.000000,27.000000,27.000000,41.000000,235.000000,10.000000,29.000000,235.000000,55.000000,80.000000,392.000000,392.000000


In [135]:
df1.shape

(51296, 26)

**~Conclusion**    
As we can see, we do not have anymore -99999 values in our first data set.

**Rows having Null values in `df2`.**

In [136]:
def count_of_null(df2, null_value):
    
    filtered_df = df2.eq(null_value)
    return filtered_df
    

In [137]:
null_count_df2 = count_of_null(df2, -99999).sum().reset_index()

In [138]:
null_count_df2.rename(columns={0: 'count'}, inplace=True)

In [153]:
null_count_df2[null_count_df2['count'] != 0].sort_values(by = 'count', ascending = False)

,index,count
47,CC_utilization,47636
49,PL_utilization,44435
3,time_since_recent_deliquency,35949
5,max_delinquency_level,35949
2,time_since_first_deliquency,35949
55,max_unsec_exposure_inPct,23178
10,max_deliq_6mts,12890
11,max_deliq_12mts,10832
34,time_since_recent_enq,6321
37,enq_L3m,6321


**`Treatment`**
- We have 21 columns with null values, `8` columns have more than `10k` null values.

**Way to treat Null values present in data**
- `Imputation` - Imputation is one way to treat null value, but in this dataset imputing values with central tendencies can create a bias in data. Because every customer have different behaviour for credit utilizatoins and deliquencies.In this case imputing can change behaviour of one customer with respect to another.

- `Deleting Columns` - We have Features with more than 35k null values, which means more than 70% data is null, These features can not play any significant role in finding correlations 

- `Deleting Rows` - If a column has less than 10k null values, then deleting rows would be a good solutions in this case as compared to others

In [154]:
columns_to_be_removed = null_count_df2[null_count_df2['count'] > 10000]['index'].tolist()

**Removing these columns from `df2`**

In [168]:
df2.drop(columns_to_be_removed, axis = 1)

In [169]:
df2.shape

(51336, 54)

**Removing Rows from those columns, with null value count less than 10k**

In [178]:
for i in df2.columns:
    df2 = df2[df2[i] != -99999]

In [179]:
df2.shape

(42066, 54)

**Now, we have successfuly removed columns and rows with large number of null values**

## Merging Datasets
**First let's find the common tables in both the tables**

In [181]:
for i in df1.columns:
    if i in list(df2.columns):
        print(i)

PROSPECTID


In [182]:
# We can now merge our dataset on PROSPECTID

In [183]:
df = pd.merge(df1, df2, how = 'inner', right_on='PROSPECTID', left_on = 'PROSPECTID')

In [184]:
df.head()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,Total_TL_opened_L12M,Tot_TL_closed_L12M,pct_tl_open_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,Auto_TL,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL,time_since_recent_payment,num_times_delinquent,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,num_times_30p_dpd,num_times_60p_dpd,num_std,num_std_6mts,num_std_12mts,num_sub,num_sub_6mts,num_sub_12mts,num_dbt,num_dbt_6mts,num_dbt_12mts,num_lss,num_lss_6mts,num_lss_12mts,recent_level_of_deliq,tot_enq,CC_enq,CC_enq_L6m,CC_enq_L12m,PL_enq,PL_enq_L6m,PL_enq_L12m,time_since_recent_enq,enq_L12m,enq_L6m,enq_L3m,MARITALSTATUS,EDUCATION,AGE,GENDER,NETMONTHLYINCOME,Time_With_Curr_Empr,pct_of_active_TLs_ever,pct_opened_TLs_L6m_of_L12m,pct_currentBal_all_TL,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,0,0,0.00,0.000,0,0,0,0,1,0,4,1,4,0,72,18,549,11,29,0,0,0,0,0,21,5,11,0,0,0,0,0,0,0,0,0,29,6,0,0,0,6,0,0,566,0,0,0,Married,12TH,48,M,51000,114,0.200,0.0,0.798,0,1,0.0,0.0,0.000,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,1,0,1.00,0.000,0,0,0,1,0,0,0,0,1,0,7,7,47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,209,1,0,0,Single,GRADUATE,23,F,19000,50,1.000,0.0,0.370,0,0,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,2,0,0.25,0.000,1,1,0,6,1,0,0,2,6,0,47,2,302,9,25,1,9,8,0,0,10,5,10,0,0,0,0,0,0,0,0,0,25,4,0,0,0,0,0,0,587,0,0,0,Married,SSC,40,M,18,191,1.000,0.5,0.585,0,0,0.0,0.0,0.000,0.0,1,0,ConsumerLoan,others,693,P2
3,5,3,2,1,0,0,0.000,0.0,0.333,0.667,0,0,0.00,0.000,0,1,0,0,0,0,0,3,0,2,131,32,583,0,0,0,0,0,0,0,53,4,16,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3951,0,0,0,Married,POST-GRADUATE,48,M,15000,75,0.333,0.0,0.000,0,0,0.0,0.0,0.000,0.0,0,0,AL,AL,753,P1
4,6,6,5,1,0,0,0.000,0.0,0.167,0.833,0,1,0.00,0.167,0,4,0,0,2,0,0,6,0,0,150,17,245,14,270,0,0,0,13,11,5,0,2,3,0,1,0,0,0,0,0,0,26,15,2,0,1,7,3,3,7,6,5,4,Married,12TH,35,M,0,154,0.167,0.0,0.000,0,0,1.0,0.0,0.429,0.0,1,0,ConsumerLoan,PL,668,P3


In [185]:
df.shape

(42064, 79)

In [187]:
df.isna().sum().sum()

0

# Feature Selection

### Categorical Variables 
We will first try to find the association between our categorical variables and Target Variable(Approved Flag)


In [207]:
cat_var = []
for i in df.columns:
    if df[i].dtype == 'object' and i != 'Approved_Flag':
        cat_var.append(i)
cat_var

['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

**Hypothesis Testing**   
- `Chi-Square` is the Test which we use to find association between Categorical - Categorical Variable.   
- We will assume `alpha` = 0.05

In [208]:
# let's calculate p-value using chi-square test

In [213]:
# Chi-Square Test
p_val = []
for i in cat_var:
    chi2, pval, _, _, = chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
    print(i, ':', pval)
    p_val.append(pval)

MARITALSTATUS : 3.578180861038862e-233
EDUCATION : 2.6942265249737532e-30
GENDER : 1.907936100186563e-05
last_prod_enq2 : 0.0
first_prod_enq2 : 7.84997610555419e-287


In [214]:
for i in p_val:
    print(p_val[0] < 0.05)

True
True
True
True
True


**As we can see calculated p-value sor all the categorical values is less than assumed alpha, we will accept all.**

### Numerical Variables 

Now we will try to find association between numerical variables and target variable with the use of appropriate Statistical Test

In [217]:
num_var = []

for i in df.columns:
    if df[i].dtype != 'object' and i not in ['PROSPECTID', 'Approved_Flag']:
        num_var.append(i)
num_var

['Total_TL',
 'Tot_Closed_TL',
 'Tot_Active_TL',
 'Total_TL_opened_L6M',
 'Tot_TL_closed_L6M',
 'pct_tl_open_L6M',
 'pct_tl_closed_L6M',
 'pct_active_tl',
 'pct_closed_tl',
 'Total_TL_opened_L12M',
 'Tot_TL_closed_L12M',
 'pct_tl_open_L12M',
 'pct_tl_closed_L12M',
 'Tot_Missed_Pmnt',
 'Auto_TL',
 'CC_TL',
 'Consumer_TL',
 'Gold_TL',
 'Home_TL',
 'PL_TL',
 'Secured_TL',
 'Unsecured_TL',
 'Other_TL',
 'Age_Oldest_TL',
 'Age_Newest_TL',
 'time_since_recent_payment',
 'num_times_delinquent',
 'max_recent_level_of_deliq',
 'num_deliq_6mts',
 'num_deliq_12mts',
 'num_deliq_6_12mts',
 'num_times_30p_dpd',
 'num_times_60p_dpd',
 'num_std',
 'num_std_6mts',
 'num_std_12mts',
 'num_sub',
 'num_sub_6mts',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_6mts',
 'num_dbt_12mts',
 'num_lss',
 'num_lss_6mts',
 'num_lss_12mts',
 'recent_level_of_deliq',
 'tot_enq',
 'CC_enq',
 'CC_enq_L6m',
 'CC_enq_L12m',
 'PL_enq',
 'PL_enq_L6m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L12m',
 'enq_L6m',
 'enq_L3m',

**Multicollinearity in Numerical features**
- We need to check if our independent Variables possess multicollinearity, if we can predict any independent feature using other independent variables, then we can not use that feature because it will wrongly interpret our model
- For ex: If we are predicting a house price, and we take two independent variable one is `Floor Area` other is `Carpet Area`, we can predict carpet area from floor area, if we use both in house price prediction it will be a misleading interpretation to the model.